In [13]:
from pyopenie import OpenIE5
import pandas as pd
import re
import sys
extractor = OpenIE5('http://localhost:8000')

In [14]:
def get_quotes(sent):

    #s = "Head Tyrus Wong, ‘Bambi’ Artist Thwarted by Racial Bias, Dies at 106"
    single_quote_pattern = r"‘(.*?)’"
    quotes_char = [r"‘", r'’']
    quotes = re.findall(single_quote_pattern, sent, re.DOTALL)

    for i in range(len(quotes_char)):
        sent = re.sub(quotes_char[i], '', sent)
        
    return sent, quotes

In [44]:
extractions = extractor.extract("This")
extractions

ConnectionError: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response',))

In [16]:
data = pd.read_csv('data/cluster_0_news_unprocessed_with_labels.csv')
sent2 = data['sent2']

In [73]:
def extract_relations(sent, tupl):
    sent, quotes = get_quotes(sent)
    print("Sentence: ", sent, "\n")
    extractions = extractor.extract(sent)
    #print(extractions)
    for ext in extractions:
        #print("ext ", ext, "\n")
        dic = dict()
        dic['confidence'] = ext['confidence']
        #tup.append(ext['extraction']['context'])
        arg1 = ext['extraction']['arg1']['text']
        arg2 = ext['extraction']['arg2s']
        rel = ext['extraction']['rel']['text']
        dic['arg1'] = arg1
        dic['rel'] = rel
        temp = []
        
        for a in arg2:
            txt = a['text']
            temp.append(txt)
            
        dic['arg2'] = temp
        tupl.append(dic)
        print("tup = ", tupl, "\n")
        
        
        if(' ' in arg1):
            extract_2 = extractor.extract(arg1)
            if(len(extract_2) > 0):
                tupl = extract_relations(arg1, tupl)
        
        for txt in temp:
            if(' ' in txt):
                extract_2 = extractor.extract(txt)
                if(len(extract_2) > 0):
                    tupl = extract_relations(txt, tupl)
        print("tuple ", tupl, "\n")
    return tupl

def get_rels(tup):
    extracted_rels = []
    for t in tup:
        temp = (t['confidence'], t['arg1'], t['rel'], t['arg2'])
        extracted_rels.append(temp)
    return extracted_rels

count = 0
extracted_rels = []
for sent in sent2[1:2]:
    #print("Sentence: ", sent, "\n")
    tup = extract_relations(sent, tupl=[])   
    extracted_rels.append(get_rels(tup))
    count+=1
    print(count)

Sentence:  [The company, which currently employs 107, 300 staff, confirmed that the first set of employees to be laid off will be informed within the next 60 days, whilst further workforce cuts will take place throughout 2017.   

tup =  [{'confidence': 0.893884626924355, 'rel': 'will be informed', 'arg2': ['within the next 60 days', 'whilst further workforce cuts will take place throughout 2017'], 'arg1': 'the first set of employees'}] 

Sentence:  whilst further workforce cuts will take place throughout 2017 

tup =  [{'confidence': 0.893884626924355, 'rel': 'will be informed', 'arg2': ['within the next 60 days', 'whilst further workforce cuts will take place throughout 2017'], 'arg1': 'the first set of employees'}, {'confidence': 0.9111779447320654, 'rel': 'will take', 'arg2': ['place', 'throughout 2017'], 'arg1': 'further workforce cuts'}] 

tuple  [{'confidence': 0.893884626924355, 'rel': 'will be informed', 'arg2': ['within the next 60 days', 'whilst further workforce cuts will t

tuple  [{'confidence': 0.893884626924355, 'rel': 'will be informed', 'arg2': ['within the next 60 days', 'whilst further workforce cuts will take place throughout 2017'], 'arg1': 'the first set of employees'}, {'confidence': 0.9111779447320654, 'rel': 'will take', 'arg2': ['place', 'throughout 2017'], 'arg1': 'further workforce cuts'}, {'confidence': 0.7987739266772069, 'rel': 'employs', 'arg2': ['107', 'currently'], 'arg1': 'The company'}, {'confidence': 0.8888883252632631, 'rel': 'confirmed', 'arg2': ['that the first set of employees to be laid off will be informed within the next 60 days, whilst further workforce cuts will take place throughout 2017'], 'arg1': 'The company, which currently employs 107, 300 staff'}, {'confidence': 0.8321465232489272, 'rel': 'employs', 'arg2': ['107', 'currently'], 'arg1': 'The company'}, {'confidence': 0.9111779447320654, 'rel': 'will take', 'arg2': ['place', 'throughout 2017'], 'arg1': 'further workforce cuts'}, {'confidence': 0.7852615174453589, 'r

In [74]:
extracted_rels

[[(0.893884626924355,
   'the first set of employees',
   'will be informed',
   ['within the next 60 days',
    'whilst further workforce cuts will take place throughout 2017']),
  (0.9111779447320654,
   'further workforce cuts',
   'will take',
   ['place', 'throughout 2017']),
  (0.7987739266772069, 'The company', 'employs', ['107', 'currently']),
  (0.8888883252632631,
   'The company, which currently employs 107, 300 staff',
   'confirmed',
   ['that the first set of employees to be laid off will be informed within the next 60 days, whilst further workforce cuts will take place throughout 2017']),
  (0.8321465232489272, 'The company', 'employs', ['107', 'currently']),
  (0.9111779447320654,
   'further workforce cuts',
   'will take',
   ['place', 'throughout 2017']),
  (0.7852615174453589, 'the first set of employees', 'to be laid off', []),
  (0.9111779447320654,
   'further workforce cuts',
   'will take',
   ['place', 'throughout 2017']),
  (0.7130135287477929, 'employees', '